In [30]:
import pathlib as pl
import pandas as pd
import statistics

pd.set_option('future.no_silent_downcasting', True)

%cd -q "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/notebooks"
_PROJECT_CONFIG_NB = str(pl.Path("00_project_config.ipynb").resolve(strict=True))

%run $_PROJECT_CONFIG_NB

summary_file = pl.Path(
    "/home/ebertp/work/projects/hgsvc/2024_gap_eval/SAMPLES.vrk-ps-sseq.t2tv2.hprc-gaps.ctg-summary-1pct.tsv"
)
summary = pd.read_csv(summary_file, sep="\t", header=0)

loc_columns = [c for c in summary.columns if c.endswith("loc_exists")]

summary["max_closed_haps"] = 130
summary.loc[summary["chrom"] == "chrY", "max_closed_haps"] = 30
summary.loc[summary["chrom"] == "chrX", "max_closed_haps"] = 100

male_columns = []
female_columns = []

for column in summary.columns:
    if not column.endswith(".sex"):
        continue
    hap_sex = summary[column].iloc[0]
    stat_column = column.replace("sex", "closed_1pct")
    if hap_sex == "male":
        male_columns.append(stat_column)
    else:
        female_columns.append(stat_column)

any_columns = male_columns + female_columns

closed_haps = []
for row in summary.itertuples():
    if row.chrom == "chrY":
        agg = (summary.loc[row.Index, male_columns] > 0).sum()
    elif row.chrom == "chrX":
        agg = (summary.loc[row.Index, female_columns] > 0).sum()
    else:
        agg = (summary.loc[row.Index, any_columns] > 0).sum()
    closed_haps.append(agg)

summary["closed_in_hap_n"] = closed_haps
summary["closed_in_hap_pct"] = (summary["closed_in_hap_n"]/summary["max_closed_haps"] * 100).round(1)

select_auto = ~summary["chrom"].isin(["chrX", "chrY"])

print(summary.loc[select_auto, "closed_in_hap_pct"].describe())    
for row in summary.itertuples():
    print(row.closed_in_hap)

count    520.000000
mean      86.339615
std       22.385166
min        0.000000
25%       88.500000
50%       95.400000
75%       97.700000
max      100.000000
Name: closed_in_hap_pct, dtype: float64


AttributeError: 'Pandas' object has no attribute 'closed_in_hap'